# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
city_table= pd.read_csv("../WeatherPy/output_data/city_data.csv")
print(len(city_table))
city_table.head()

575


,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,kroya,-7.6331,109.2461,86.79,57,4,12.62,ID,1618900483
1,arraial do cabo,-22.9661,-42.0278,73.90,78,30,27.31,BR,1618900487
2,cabo san lucas,22.8909,-109.9124,69.01,85,36,7.18,MX,1618900448
3,longyearbyen,78.2186,15.6401,28.40,69,75,12.66,SJ,1618900494
4,ancud,-41.8697,-73.8203,46.99,100,0,2.84,CL,1618900497


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
coordinates = city_table[["Lat", "Lon"]]
locations = city_table["Humidity"].astype(float)

fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(coordinates, weights = locations, dissipating=False, point_radius= 5)
fig.add_layer(heatmap)
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
hoteldf = pd.DataFrame()
perfect_weather_city = city_table.loc[(city_table["Max_Temp"] > 70) & (city_table["Max_Temp"] < 78) & 
                                  (city_table["Wind_Speed"] < 10) & 
                                  (city_table["Cloudiness"] == 0)]
perfect_weather_city

,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
7,rikitea,-23.1203,-134.9692,77.97,69,0,8.05,PF,1618900459
63,taoudenni,22.6783,-3.9836,77.45,13,0,8.41,ML,1618900712
225,aquiraz,-3.9014,-38.3911,77.00,94,0,2.30,BR,1618901337
290,changji,44.0167,87.3167,75.20,13,0,4.47,CN,1618901596
372,fereydun kenar,36.6864,52.5225,73.40,56,0,6.91,IR,1618901914


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:

coordinates = perfect_weather_city[["Lat", "Lon", "City", "Country"]]

for index, row in coordinates.iterrows():

    target_coordinates = f"{row.Lat}, {row.Lon}"
    target_search = "Hotel"
    radius = 5000
    
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": radius,
    "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    places_data = response.json()
    try:
        hoteldf.loc[index, "Hotel_Name"] = places_data["results"][0]["name"]
        hoteldf.loc[index, "Lat"] = row.Lat
        hoteldf.loc[index, "Lon"] = row.Lon
        hoteldf.loc[index, "City_Name"] = row.City
        hoteldf.loc[index, "Country"] = row.Country
        print(f"A Hote for {row.City} was found")
    except:
        print('No Hotel Was Found')
        pass
hoteldf

No Hotel Was Found
No Hotel Was Found
A Hote for aquiraz was found
A Hote for changji was found
A Hote for fereydun kenar was found


,Hotel_Name,Lat,Lon,City_Name,Country
225,Brisa do Golf (Golf Ville Resort),-3.9014,-38.3911,aquiraz,BR
290,Garden Hotel,44.0167,87.3167,changji,CN
372,هتل اپارتمان بهشت motel paradise,36.6864,52.5225,fereydun kenar,IR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City_Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hoteldf.iterrows()]
locations = hoteldf[["Lat", "Lon"]]

In [9]:
# Add marker layer ontop of heat map
markers =gmaps.marker_layer(locations, info_box_content = hotel_info, display_info_box =True)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))